<!---
The MIT License (MIT)

Copyright (c) 2017 Sean UN Wood

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
--->

# Real-time Speech Enhancement with GCC-NMF

#### Sean UN Wood, August 2017

In this iPython notebook, we present a real-time Python implementation of the GCC-NMF speech enhancement algorithm presented in:

- Sean UN Wood and Jean Rouat, [*Real-time Speech Enhancement with GCC-NMF: Demonstration on the Raspberry Pi and NVIDIA Jetson*](https://www.researchgate.net/profile/Sean_Wood7/publication/318946628_Real-time_Speech_Enhancement_with_GCC-NMF_Demonstration_on_the_Raspberry_Pi_and_NVIDIA_Jetson/links/59872715aca27266ada22465/Real-time-Speech-Enhancement-with-GCC-NMF-Demonstration-on-the-Raspberry-Pi-and-NVIDIA-Jetson.pdf), **Interspeech 2017 Show and Tell Demonstrations**.

that built on previous work:
 - Sean UN Wood and Jean Rouat, [*Real-time Speech Enhancement with GCC-NMF*](https://www.researchgate.net/publication/318511757_Real-time_Speech_Enhancement_with_GCC-NMF), **Interspeech 2017**.

This is essentially a real-time implementation of the [Online Speech Enhancement](https://nbviewer.jupyter.org/github/seanwood/gcc-nmf/blob/master/notebooks/onlineSpeechEnhancement.ipynb) notebook presented previously, with the addition of soft GCC-NMF mask generation.


## Notebook Overview

1. Preliminary setup: Python dependencies
2. Real-time GCC-NMF: Graphical User Interface (GUI)
3. Real-time GCC-NMF: Command-line Interface (CLI)
4. Configuration parameters

# 1. Preliminary setup
### Dependencies
In addition to [numpy](http://www.numpy.org/) and [scipy](https://www.scipy.org/), a few additional dependencies are required:

1. [Theano](http://deeplearning.net/software/theano/) for its GPU acceleration and optimizing compiler.
2. [PyAudio](https://people.csail.mit.edu/hubert/pyaudio/) for real-time audio playback.

To run the graphical user interface, we also require:
1. [PyQt](https://riverbankcomputing.com/software/pyqt/intro) Python bindings for the [Qt](https://www.qt.io/) application framework.
2. [pyqtgraph](http://www.pyqtgraph.org/) scientific graphics and GUI library.

### Installing dependencies
All dependencies can be installed with [pip](https://pip.pypa.io/en/stable/), either on the command line:

`$ pip install numpy scipy theano pyaudio`

`$ pip install pyqt5 pyqtgraph`

or programatically from within this notebook or a Python interpreter:

In [1]:
import pip

In [2]:
# required dependencies
pip.main(['install', 'numpy'])
pip.main(['install', 'scipy'])
pip.main(['install', 'theano'])
pip.main(['install', 'pyaudio'])
print('\nFinished installing required dependencies')


Finished installing required dependencies


In [3]:
# dependencies for GUI
pip.main(['install', 'pyqt5'])
pip.main(['install', 'pyqtgraph'])
print('\nFinished installing GUI dependencies')


Finished installing GUI dependencies


# 2. Real-time GCC-NMF: with GUI

#### Preliminary imports for logging

In [4]:
import logging
logging.getLogger().setLevel(logging.INFO)

#### Running real-time GCC-NMF

The real-time GCC-NMF executable can be found at `gccNMF/realtime/runRealtimeGCCNMF.py`. At the root directory of this repo, we can start the program as follows:

`$ python gccNMF/realtime/runRealtimeGCCNMF.py`

We can also start it programatically:

In [5]:
from gccNMF.realtime.runRealtimeGCCNMF import RealtimeGCCNMF
RealtimeGCCNMF()
print('Done!')

GCCNMFConfig: loading configuration params...
TDOA
    numTDOAs: 64
    numTDOAHistory: 128
    numSpectrogramHistory: 128
    gccPHATNLAlpha: 2.0
    gccPHATNLEnabled: False
    microphoneSeparationInMetres: 0.1
    targetTDOAEpsilon: 5.0
    targetTDOABeta: 2.0
    targetTDOANoiseFloor: 0.0
Audio
    numChannels: 2
    sampleRate: 16000
    deviceIndex: None
STFT
    windowSize: 1024
    hopSize: 512
    blockSize: 512
NMF
    dictionarySize: 64
    dictionarySizes: [64, 128, 256, 512, 1024]
    dictionaryType: Pretrained
    numHUpdates: 0
GCCNMFPretraining: Loading pretrained W (size 64): /gcc-nmf/data/pretrainedW/W_64.npy
GCCNMFPretraining: Pretrained W not found at /gcc-nmf/data/pretrainedW/W_64.npy, creating...
GCCNMFPretraining: Loading pretrained W (size 128): /gcc-nmf/data/pretrainedW/W_128.npy
GCCNMFPretraining: Pretrained W not found at /gcc-nmf/data/pretrainedW/W_128.npy, creating...
GCCNMFPretraining: Loading pretrained W (size 256): /gcc-nmf/data/pretrainedW/W_256.npy
GC

Startup may take a little while the first time it runs, as we pre-learn several NMF dictionaries of varying size. The dictionaries are saved to `data/realtimeGCCNMFPretrainedW`, so subsequent launches will be much quicker.

Once the window opens, click the `Play` button (or hit space) and you should see something like this:

![Real-time GCC-NMF Interface](images/realtimeGCCNMFScreenshot.png "Real-time GCC-NMF Interface")

The images roll with the input audio in waterfall-style. At the left, we see the input spectrogram on top, and the output spectrogram on the bottom. At the right, we have the GCC-PHAT angular spectrogram of the input on top (numTDOA x time), and the NMF mask on the bottom (numAtoms x time). In the center-bottom, we see the currently selected NMF dictionary (numAtom x frequency). Finally, at the center-top, we have the controls over the GCC-NMF masking function, dictionary size, number of coefficient inference updates per frame, as well as buttons to control playback, enable/disable enhancement, and toggle the info strings above the images.

# 3. Real-time GCC-NMF: no GUI

Real-time GCC-NMF can also be run without the GUI, by passing the `--no-gui` as an argument on the command line:

`$ python gccNMF/realtime/runRealtimeGCCNMF.py --no-gui`

or programatically by instantiating the `RealtimeGCCNMFNoGUI` class instead of the `RealtimeGCCNMF` class we used above:

In [ ]:
from gccNMF.realtime.runRealtimeGCCNMF import RealtimeGCCNMFNoGUI
RealtimeGCCNMFNoGUI()
print('Done!')

# 4. Configuration parameters

#### Input and Config files
GCC-NMF parameters can be set via a configuration file, with a default config file generated as `gccNMF.config` at first launch. This config file can either be modified, or a new config file may be specified. The input wav file may also be specified in a similar manner.

The config and input file paths can either be specified with the following optional command line arguments,

`$ python gccNMF/realtime/runRealtimeGCCNMF.py --input <path/to/wav/file> --config <path/to/config/file>`

or programatically as,

`RealtimeGCCNMF(audioPath='<path/to/wav/file>', configPath='<path/to/config/file>')`

#### Help

`$ python gccNMF/realtime/runRealtimeGCCNMF.py --help`

`usage: runRealtimeGCCNMF.py [-h] [-i INPUT] [-c CONFIG] [--no-gui]`

`Real-time GCC-NMF Speech Enhancement`

`optional arguments:`

`-h, --help`  
*show this help message and exit*  

`-i INPUT, --input INPUT`  
*input wav file path*  

`-c CONFIG, --config CONFIG`  
*config file path*  

`--no-gui`  
*no user interface mode*